In [ ]:
import os
import cv2
import tensorflow as tf

In [ ]:
import wget

In [ ]:
PRETRAINED_MODEL_NAME = "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"
PRETRAINED_MODEL_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz"

In [ ]:
#setting the needed paths
paths = {
    'WORKSPACE_PATH': os.path.join('hackathon', 'workspace'),
    'MODEL_API_PATH': os.path.join('hackathon','models'),
    'PROTOC_PATH': os.path.join('hackathon','protoc'),
    'PRETRAINED_MODEL_PATH': os.path.join('hackathon','workspace', 'pre-trained_model'),
    'IMAGE_PATH': os.path.join('hackathon','workspace', 'images'),
    'TEST_IMAGE_PATH': os.path.join('hackathon', 'workspace', 'images', 'test')
    }

In [ ]:
PIPELINE_CONFIG = os.path.join('hackathon', 'workspace','model', CUSTOM_MODEL_NAME, 'pipeline.config')

In [ ]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [ ]:
if not os.path.exists(os.path.join(paths['MODEL_API_PATH'], 'research', 'object_detection')):
    #this will clone the model api to our specified model path
    !git clone https://github.com/tensorflow/models {paths['MODEL_API_PATH']}

In [ ]:
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !mv protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && unzip protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd hackathon/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py setup.py && python setup.py build && python setup.py install
    !cd hackathon/models/research/slim && pip install -e .

## Trained Model Loading

In [ ]:
import object_detection
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils
from object_detection.builders import model_builder

In [ ]:
# building the detection model
config = config_util.get_configs_from_pipeline_file(os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config'))
model = model_builder.build(model_config=config['model'], is_training=False)

In [ ]:
# restore checkpoint from disk to use for detection
checkpoint= tf.compat.v2.train.Checkpoint(model)
checkpoint_path = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
checkpoint.restore(checkpoint_path).expect_partial() # expect_partial() to avoid warning of unused checkpoint indexes

In [ ]:
@tf.function #converts the code below to tensorflow graph function for better performance
def detection_fn(image):
    image, shape = model.preprocess(image)
    prediction = model.predict(image, shape)
    detection = model.postprocess(prediction, shape)
    return detection

## Label map loading

In [ ]:
LABELMAP_NAME = "mscoco_label_map.pbtxt"

In [ ]:
label_map = label_map_util.load_labelmap(os.path.join(paths['MODEL_API_PATH'], 'research', 'object_detection', 'data', LABELMAP_NAME))

In [ ]:
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)

In [ ]:
category_index = label_map_util.create_category_index(categories)

## Detecting from an image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from object_detection.utils import label_map_util
%matplotlib inline

In [ ]:
IMAGE_PATH = os.path.join(paths['TEST_IMAGE_PATH'], '15.avi_snapshot_05.57_[2021.06.22_02.34.23].jpg')

In [ ]:
#print(image[0][:10])

In [ ]:
#print(result_dict)

In [ ]:
#print(boxes)

In [ ]:
#print(scores)

In [ ]:
#print(classes)

In [ ]:
#print(result_dict['num_detections'])

In [ ]:
image = cv2.imread(IMAGE_PATH)
converted_img = np.array(image)

input_tensor = tf.convert_to_tensor(np.expand_dims(converted_img, 0), dtype=tf.float32)

detections = detection_fn(input_tensor)
num_detections = int(detections.pop('num_detections'))

detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
detections['num_detections'] = num_detections
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

boxes = detections['detection_boxes']
classes = detections['detection_classes']
scores = detections['detection_scores']


vis_utils.visualize_boxes_and_labels_on_image_array(
    converted_img,
    boxes,
    classes+1,
    scores,
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=10,
    min_score_thresh=0,
    line_thickness=5,
    agnostic_mode=False
    )

rect_image = cv2.cvtColor(converted_img, cv2.COLOR_BGR2RGB)
plt.imshow(rect_image)
plt.show()